In [1]:
import sys
#sys.path.insert(0,'/home/lshtanko/Programming/another-fem-framework/')
sys.path.insert(0,'/Users/marusy/Programming/another-fem-framework/')
#sys.path.insert(0,'/Users/leonshting/Programming/Schlumberger/fem-framework/')

In [2]:
#import h5py

from operator_assembler.assembly_interface import AssemblyInterface2D
from operator_assembler.n_to_1_ddof_allocator import Nto1Allocator2D
from grid.grid_manager import GridManager
from grid.allocator_interface import InterfaceDofAllocator2D
from grid.grid_domain import GridDomain
from operator_assembler.matrix_assembler import MatrixAssembler2D
from scipy.ndimage import imread
from grid import cell, grid_layer, grid_manager
import h5py
import numpy as np

from common import visual
from common.tests import *
from common.helpers import *
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse import linalg

%matplotlib inline

#### Bad cell

In [ ]:
data_start = (16, 20)
data_shape = (4, 4)
data_end = tuple([i+j for i,j in zip(data_start, data_shape)])

h5_data = h5py.File('../../../GM_L3D/600/state/viz_GM_1.h5')['index'][0]
data = h5_data[data_start[0]:data_end[0], data_start[1]:data_end[1]]
#data = imread('/Users/marusy/Programming/model/bhi2_labelled0000.tif')[data_start[0]:data_end[0], data_start[1]:data_end[1]]

grid_domain = GridDomain(integer_size=data_shape, domain_size=(1.,1.))
gm = GridManager()
ifma = InterfaceDofAllocator2D(grid_manager=gm.fit(data=data))

gm.draw_grid()

In [3]:
cell1 = cell.Cell2D(size=(1,1), ll_vertex=(0,0))
cell2 = cell.Cell2D(size=(1,1), ll_vertex=(0,1))

cell3 = cell.Cell2D(size=(2,2), ll_vertex=(1,0))

In [4]:
layer1 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell1, cell2]], layer_number=0, index=[1,1], div_index=[1,1])
layer2 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell3]], layer_number=1, index=[1], div_index=[1])

In [5]:
gm = grid_manager.GridManager(dim=2, method='coarse2to1')
gm.grid_layers = [layer1, layer2]
ifma = InterfaceDofAllocator2D(grid_manager=gm)

In [6]:
ma = Nto1Allocator2D(grid_interface=ifma)
ma._make_ddof_index()
grid_domain = GridDomain(domain_size=(3.,2.), integer_size=(3,2))
grid_domain.make_pointwise_index(ma)

In [7]:
ifma2 = AssemblyInterface2D(allocator=ma)
MA = MatrixAssembler2D(assembly_interface=ifma2, grid_domain=grid_domain)
deep = MA.assemble()

[(0, 10), (1, 6), (2, 7), (3, 8), (4, 9), (5, 4), (6, 11), (7, 20), (8, 21), (9, 22), (10, 23), (11, 0), (12, 12), (13, 24), (14, 25), (15, 26), (16, 27), (17, 1), (18, 13), (19, 28), (20, 29), (21, 30), (22, 31), (23, 2), (24, 14), (25, 32), (26, 33), (27, 34), (28, 35), (29, 3), (30, 15), (31, 16), (32, 17), (33, 18), (34, 19), (35, 5)] [(0, 4), (1, 36), (2, 37), (3, 38), (4, 39), (5, 40), (6, 0), (7, 50), (8, 51), (9, 52), (10, 53), (11, 41), (12, 1), (13, 54), (14, 55), (15, 56), (16, 57), (17, 42), (18, 2), (19, 58), (20, 59), (21, 60), (22, 61), (23, 43), (24, 3), (25, 62), (26, 63), (27, 64), (28, 65), (29, 44), (30, 5), (31, 46), (32, 47), (33, 48), (34, 49), (35, 45)]
[(0, 10), (1, 6), (2, 7), (3, 8), (4, 9), (5, 4), (6, 11), (7, 20), (8, 21), (9, 22), (10, 23), (11, 0), (12, 12), (13, 24), (14, 25), (15, 26), (16, 27), (17, 1), (18, 13), (19, 28), (20, 29), (21, 30), (22, 31), (23, 2), (24, 14), (25, 32), (26, 33), (27, 34), (28, 35), (29, 3), (30, 15), (31, 16), (32, 17), (3

In [ ]:
sine_test = grid_domain.vectorize_function(lambda x,y: np.sin(x+y))
product = grid_domain.devectorize_vector(MA.assembled * sine_test)
init = grid_domain.devectorize_function(lambda x,y: np.sin(x+y))

visual.plot_surface_unstructured_w_dict(
    point_val_dict=grid_domain.devectorize_vector(MA.assembled * sine_test),
    plot_domain_shape=grid_domain.domain_size,
    int_domain_shape=grid_domain.integer_size
)

visual.plot_surface_unstructured_w_dict(
    point_val_dict=grid_domain.devectorize_function(lambda x,y: np.sin(x+y)),
    plot_domain_shape=grid_domain.domain_size,
    int_domain_shape=grid_domain.integer_size
)

In [8]:
def partial_diff_test2(pointnum, point, matrix, pointdict, mass_matrix=None,
                      multiple_dofs_on_point=True):
    x_der = 0
    xx_der = 0
    yy_der = 0
    y_der = 0
    xy_der = 0

    if not multiple_dofs_on_point:
        for k, v in pointdict.items():
            dx = point[0] - k[0]
            dy = point[1] - k[1]

            x_der += dx * matrix[pointnum, v]
            xx_der += matrix[pointnum, v] * dx ** 2 / 2
            y_der += dy * matrix[pointnum, v]
            yy_der += dy ** 2 / 2 * matrix[pointnum, v]
            xy_der += dx * dy * matrix[pointnum, v]
    else:
        for k, vs in pointdict.items():
            for v in vs:
                dx = point[0] - k[0]
                dy = point[1] - k[1]

                x_der += dx * matrix[pointnum, v]
                xx_der += matrix[pointnum, v] * dx ** 2 / 2
                y_der += dy * matrix[pointnum, v]
                yy_der += dy ** 2 / 2 * matrix[pointnum, v]
                xy_der += dx * dy * matrix[pointnum, v]

    if mass_matrix is not None:
        print('Mass coef: {}'.format(mass_matrix[pointnum].sum()))

    print('X derivative: {}'.format(x_der))
    print('Y derivative: {}'.format(y_der))
    print('XX derivative: {}'.format(xx_der))
    print('YY derivative: {}'.format(yy_der))
    print('XY_derivative: {}'.format(xy_der), end='\n\n')
    return (xx_der + yy_der)/2

In [9]:
mm = np.zeros(MA.half_glob.shape[0])
for i in grid_domain.normal_index.keys():
    print(i, end=' ')
    mm[i] = partial_diff_test2(matrix=MA.half_glob, point=grid_domain.normal_index[i], pointnum=i, pointdict=grid_domain.pointwise_index)

10 X derivative: 0.03333541346427044
Y derivative: 0.033332398026687984
XX derivative: -0.0011122795970912557
YY derivative: -0.0011117218599699386
XY_derivative: -1.4974450391197491e-06

6 X derivative: 0.1892386240847961
Y derivative: -6.444631540092338e-07
XX derivative: -0.006308282187703948
YY derivative: -0.006306627010608329
XY_derivative: 1.5451975784730454e-06

7 X derivative: 0.2774280501773063
Y derivative: 4.316413122928964e-06
XX derivative: -0.00924672371245952
YY derivative: -0.009249120129202829
XY_derivative: -1.9650028870166203e-06

8 X derivative: 0.27743000242700083
Y derivative: 2.3538138435585942e-06
XX derivative: -0.00924779957618638
YY derivative: -0.009245984558303354
XY_derivative: -3.415722819311018e-07

9 X derivative: 0.18923905133989335
Y derivative: 1.6958989661544632e-07
XX derivative: -0.006308738820388876
YY derivative: -0.006307776805794943
XY_derivative: 4.890466017495876e-07

4 X derivative: 0.06667102695146793
Y derivative: -3.0894642219919666e-06

XX derivative: -0.006307399145926609
YY derivative: -0.006307159567218617
XY_derivative: -1.3286725468464322e-06

54 X derivative: -3.172581774291139e-06
Y derivative: -1.2526182520823659e-05
XX derivative: -0.05249751913938125
YY derivative: -0.05249637124004264
XY_derivative: 6.122052767509878e-09

55 X derivative: 1.0442415556102813e-05
Y derivative: -5.175549287257664e-06
XX derivative: -0.07696796358781717
YY derivative: -0.07696563315796723
XY_derivative: -2.4224981643727883e-06

56 X derivative: -3.19421755520799e-06
Y derivative: -1.4970059701926997e-07
XX derivative: -0.07696734871901001
YY derivative: -0.07696910442304053
XY_derivative: 5.889514417180969e-07

57 X derivative: -3.0712539185739907e-06
Y derivative: 1.8104632711207769e-06
XX derivative: -0.05249930101645193
YY derivative: -0.052496067484219915
XY_derivative: -1.928881387778733e-06

42 X derivative: 1.7902093594340807e-07
Y derivative: -0.27743224514658743
XX derivative: -0.009247894094025213
YY derivative: -0.00

98 X derivative: -1.38841669604442e-05
Y derivative: 1.5394711402522132e-05
XX derivative: -0.1432577262054948
YY derivative: -0.1432540186609689
XY_derivative: 1.9054163850160122e-05

99 X derivative: -3.6426477695652992e-06
Y derivative: 1.7186003770097713e-07
XX derivative: -0.21000367898273162
YY derivative: -0.20999918692708153
XY_derivative: 8.152491995788713e-07

100 X derivative: -1.1191200803561951e-07
Y derivative: -5.519020702532407e-06
XX derivative: -0.21000383706683973
YY derivative: -0.21000401173279246
XY_derivative: -5.449917082005207e-06

101 X derivative: 8.004345783130251e-07
Y derivative: 1.9387009948107492e-07
XX derivative: -0.14324284581386565
YY derivative: -0.1432425395223318
XY_derivative: 5.1877775696825e-06

69 X derivative: 1.4972284138742298e-06
Y derivative: -0.378474576105767
XX derivative: -0.0252297830277473
YY derivative: -0.02523140190432651
XY_derivative: -4.5761378577105494e-08

76 X derivative: -0.06667062239263431
Y derivative: 0.066667136671539

In [10]:
MM = csr_matrix(np.diag(-1/mm))

In [20]:
for i in sorted(grid_domain.normal_index.keys()):
    print(i, end=' ')
    partial_diff_test(matrix=deep[0][1].T * MA.half_glob, point=grid_domain.normal_index[i], pointnum=i, pointdict=grid_domain.pointwise_index)

0 X derivative: -4.726722500902891e-07
Y derivative: -4.6669244690988876e-07
XX derivative: -0.012615285857345893
YY derivative: -0.01261487236397147
XY_derivative: -3.716735108572067e-07

1 X derivative: 3.544971872909697e-06
Y derivative: 9.077965097692431e-07
XX derivative: -0.01849696686310434
YY derivative: -0.018498515633941573
XY_derivative: -2.179327529494568e-06

2 X derivative: -2.6738619019264775e-06
Y derivative: 2.8303634221349405e-06
XX derivative: -0.018495294844009125
YY derivative: -0.018497327736229134
XY_derivative: 1.651207158087642e-07

3 X derivative: -4.383479700331705e-07
Y derivative: -1.064384269611196e-06
XX derivative: -0.012615280428461233
YY derivative: -0.012581958072329424
XY_derivative: -2.109558053242644e-06

4 X derivative: 0.06667102695146793
Y derivative: -3.0894642219919666e-06
XX derivative: -0.002224313167352082
YY derivative: -0.0022237886536749622
XY_derivative: 1.2740150483192547e-07

5 X derivative: -1.3751399647654594e-05
Y derivative: -5.94

XX derivative: -0.00945624314402721
YY derivative: -0.36089013667689007
XY_derivative: 6.017266858948656e-07

47 X derivative: 7.676786767496502e-06
Y derivative: -0.05375965192146448
XX derivative: -0.013880923255730315
YY derivative: 0.6557748253244933
XY_derivative: -8.344458875875205e-06

48 X derivative: -1.8532392104253503e-06
Y derivative: 0.055986821873131726
XX derivative: -0.013870853326341248
YY derivative: -0.18526825883460385
XY_derivative: -6.197163206211886e-08

49 X derivative: 3.4423796667175377e-06
Y derivative: -0.016315986782152684
XX derivative: -0.009464319915440417
YY derivative: 0.04816147466021035
XY_derivative: 3.6847196265240613e-06

50 X derivative: 6.173400770557308e-06
Y derivative: -4.904306141657183e-06
XX derivative: -0.03581344541401648
YY derivative: -0.035809845762738844
XY_derivative: -4.039145772053235e-06

51 X derivative: -4.64707644148854e-06
Y derivative: 4.139267396729984e-06
XX derivative: -0.05249801636016607
YY derivative: -0.05250310879443

93 X derivative: -8.216662401555608e-06
Y derivative: -4.534607192189372e-06
XX derivative: -0.21000123097846807
YY derivative: -0.2100003762318032
XY_derivative: -5.9266555086159856e-06

94 X derivative: 6.2215820180386294e-06
Y derivative: -1.0839057523918255e-05
XX derivative: -0.20999420995628496
YY derivative: -0.2099940992471635
XY_derivative: -1.512482096541895e-06

95 X derivative: 9.199295289384624e-07
Y derivative: 4.68884893096988e-06
XX derivative: -0.30786502285672535
YY derivative: -0.3078680502356916
XY_derivative: 4.276342112488618e-06

96 X derivative: -1.6162808449033533e-05
Y derivative: 5.574454970819209e-06
XX derivative: -0.30787672755354556
YY derivative: -0.3078676555420628
XY_derivative: 1.2344459718559297e-06

97 X derivative: -5.660032509624546e-06
Y derivative: -9.463607436751426e-06
XX derivative: -0.21000121260286622
YY derivative: -0.21000659214784526
XY_derivative: -5.927399324489463e-06

98 X derivative: -1.38841669604442e-05
Y derivative: 1.53947114025

In [14]:
for i in grid_domain.normal_index.keys():
    print(i, end=' ')
    partial_diff_test(matrix=MA.assembled, point=grid_domain.normal_index[i], pointnum=i, pointdict=grid_domain.pointwise_index)

10 X derivative: 0.03333541346427043
Y derivative: 0.033332398026687984
XX derivative: -0.0011122795970912557
YY derivative: -0.0011117218599699524
XY_derivative: -1.4974450391197491e-06

6 X derivative: 0.18923862408479614
Y derivative: -6.444631539814782e-07
XX derivative: -0.006308282187703941
YY derivative: -0.006306627010608329
XY_derivative: 1.5451975784730454e-06

7 X derivative: 0.2774280501773063
Y derivative: 4.316413122928964e-06
XX derivative: -0.00924672371245952
YY derivative: -0.009249120129202829
XY_derivative: -1.9650028870166203e-06

8 X derivative: 0.2774300024270009
Y derivative: 2.3538138435447165e-06
XX derivative: -0.00924779957618638
YY derivative: -0.009245984558303354
XY_derivative: -3.415722819033462e-07

9 X derivative: 0.18923905133989327
Y derivative: 1.6958989667182484e-07
XX derivative: -0.006308738820388857
YY derivative: -0.006307776805794935
XY_derivative: 4.890466016932091e-07

4 X derivative: 0.0666710269514678
Y derivative: -3.0894642220474777e-06


XY_derivative: 4.198539978536239e-06

41 X derivative: -1.0997347858754392e-06
Y derivative: -0.18923904777883133
XX derivative: -0.006307399145926609
YY derivative: -0.006307159567218604
XY_derivative: -1.3286725468308197e-06

54 X derivative: -3.172581774277261e-06
Y derivative: -1.2526182520858353e-05
XX derivative: -0.052497519139381235
YY derivative: -0.05249637124004265
XY_derivative: 6.122052781387666e-09

55 X derivative: 1.0442415556144446e-05
Y derivative: -5.175549287257664e-06
XX derivative: -0.07696796358781718
YY derivative: -0.07696563315796723
XY_derivative: -2.4224981643589105e-06

56 X derivative: -3.19421755520799e-06
Y derivative: -1.4970059704702554e-07
XX derivative: -0.07696734871901001
YY derivative: -0.07696910442304053
XY_derivative: 5.889514417180969e-07

57 X derivative: -3.071253918546235e-06
Y derivative: 1.8104632712491464e-06
XX derivative: -0.05249930101645192
YY derivative: -0.05249606748421983
XY_derivative: -1.928881387785672e-06

42 X derivative: 1.

XX derivative: -0.1432577262054948
YY derivative: -0.1432540186609689
XY_derivative: 1.9054163850215633e-05

99 X derivative: -3.6426477693432546e-06
Y derivative: 1.7186003770097713e-07
XX derivative: -0.2100036789827315
YY derivative: -0.20999918692708153
XY_derivative: 8.15249199537238e-07

100 X derivative: -1.1191200803561951e-07
Y derivative: -5.519020702587918e-06
XX derivative: -0.21000383706683978
YY derivative: -0.21000401173279246
XY_derivative: -5.449917082005207e-06

101 X derivative: 8.004345780909805e-07
Y derivative: 1.9387009942556377e-07
XX derivative: -0.14324284581386576
YY derivative: -0.14324253952233185
XY_derivative: 5.187777569571478e-06

69 X derivative: 1.4972284137632075e-06
Y derivative: -0.3784745761057671
XX derivative: -0.0252297830277473
YY derivative: -0.025231401904326538
XY_derivative: -4.5761378688127796e-08

76 X derivative: -0.0666706223926342
Y derivative: 0.06666713667153945
XX derivative: -0.0044481938995198644
YY derivative: -0.004444636260071

In [ ]:
MA.I_s2b

In [ ]:
MA.half_glob[2].toarray().flatten()

In [ ]:
deep[0][1].toarray()[[2, 7, 6]]

In [ ]:
deep[0][1][2].toarray()

In [ ]:
deep[0][1][:,16].toarray()

In [ ]:
MA.assembled[2].toarray().flatten()[-9:]

In [ ]:
pipka[2].toarray().flatten()

In [ ]:
partial_diff_test(matrix=deep[0][-1], point=grid_domain.normal_index[2], pointnum=2, pointdict=grid_domain.pointwise_index)

In [ ]:
dist = distributed_eye_easy(ma.get_cell_list_of_ddofs(cell=cell3), axis2shape=MA.assembly_interface.get_ddof_count())

In [ ]:
pipka = dist.T * csr_matrix(MA._get_local_gg_matrix(distribution='lobatto', order=2)) * dist

In [ ]:
mm = np.array(MA.assembled_mass.sum(axis=1)).flatten()

In [ ]:
mm_sp = csr_matrix(np.diag(1/mm))

In [ ]:
op =  mm_sp * MA.assembled 

In [12]:
grid_domain.normal_index

{0: (0.11747235059738159, 1.0),
 1: (0.35738423466682434, 1.0),
 2: (0.64261579513549805, 1.0),
 3: (0.88252764940261841, 1.0),
 4: (0.0, 1.0),
 5: (1.0, 1.0),
 6: (0.0, 0.11747235059738159),
 7: (0.0, 0.35738423466682434),
 8: (0.0, 0.64261579513549805),
 9: (0.0, 0.88252764940261841),
 10: (0.0, 0.0),
 11: (0.11747235059738159, 0.0),
 12: (0.35738423466682434, 0.0),
 13: (0.64261579513549805, 0.0),
 14: (0.88252764940261841, 0.0),
 15: (1.0, 0.0),
 16: (1.0, 0.11747235059738159),
 17: (1.0, 0.35738423466682434),
 18: (1.0, 0.64261579513549805),
 19: (1.0, 0.88252764940261841),
 20: (0.11747235059738159, 0.11747235059738159),
 21: (0.11747235059738159, 0.35738423466682434),
 22: (0.11747235059738159, 0.64261579513549805),
 23: (0.11747235059738159, 0.88252764940261841),
 24: (0.35738423466682434, 0.11747235059738159),
 25: (0.35738423466682434, 0.35738423466682434),
 26: (0.35738423466682434, 0.64261579513549805),
 27: (0.35738423466682434, 0.88252764940261841),
 28: (0.64261579513549

In [ ]:
MA.assembly_interface.allocator.get_cell_list_of_ddofs(cell2)

In [ ]:
plt.imshow(deep[0][-1].toarray(), interpolation='none')

In [ ]:
f = plt.figure(figsize=(10,10))
plt.imshow(deep[0][1].toarray(), interpolation='none')

In [ ]:
partial_diff_test(matrix=MA.assembled, point=grid_domain.normal_index[85], pointnum=85, pointdict=grid_domain.pointwise_index)

In [ ]:
partial_diff_test(matrix=MA.assembled, point=grid_domain.normal_index[19], pointnum=19, pointdict=grid_domain.pointwise_index)

In [ ]:
partial_diff_test(matrix=MA.half_glob, point=grid_domain.normal_index[85], pointnum=85, pointdict=grid_domain.pointwise_index)

In [ ]:
partial_diff_test(matrix=MA.half_glob, point=grid_domain.normal_index[19], pointnum=19, pointdict=grid_domain.pointwise_index)

In [ ]:
csr_matrix(MA._get_local_gg_matrix(distribution='lobatto', order=2))

In [ ]:
from common import polynom_factory

In [ ]:
mmatrix = polynom_factory.local_funcfunc_matrix(distribution='globatto', order=2, dim=2, size=(0,1))

In [ ]:
tomul = np.diag(1/mmatrix.sum(axis=1))

In [ ]:
matrix, index = polynom_factory.local_gradgrad_matrix(distribution='globatto', order=2, dim=2)

In [ ]:
for point, num in index.items():
    partial_diff_test(matrix=np.dot(tomul, matrix), point=point, pointnum=num, pointdict=index, multiple_dofs_on_point=False)

In [ ]:
for point, num in index.items():
    partial_diff_test(matrix=matrix, point=point, pointnum=num, pointdict=index, multiple_dofs_on_point=False)